# Run the model

You must run this notebook only if the notebook [`01-prepare-data.ipynb`](./01-prepare-data.ipynb) has been ran.
We rely on the existence of the pre-processed dataset files to run the model.

First, we load the dataset from the CSV

In [4]:
import pandas as pd

data = pd.read_csv('data/reduced_dataset.csv')
print(data.head())

   Seating Capacity  Average Meal Price  Weekend Reservations  \
0                38               73.98                    13   
1                76               28.11                    48   
2                48               48.29                    27   
3                34               51.55                     9   
4                88               75.98                    37   

   Weekday Reservations     Revenue  Marketing&Followers  Location_Downtown  \
0                     4   638945.52            -0.870544              False   
1                     6   490207.83             0.712762               True   
2                    14   541368.62            -0.122085              False   
3                    17   404556.80            -1.591078              False   
4                    26  1491046.35             0.314123               True   

   Location_Rural  Location_Suburban  Cuisine_American  Cuisine_French  \
0            True              False             False      

After loading the data, we need to perform some immediate pre-processing. We cannot (in a simple way) load the already scaled values from the dataset, as we'll not be able to descale them back (in a simple way), so let's scale the columns right here.

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# list of columns ONLY for the REDUCED dataset
columns_to_normalize = ['Revenue', 'Seating Capacity', 'Average Meal Price', 'Weekend Reservations', 'Weekday Reservations']
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
print(data.head())


   Seating Capacity  Average Meal Price  Weekend Reservations  \
0         -1.276714            1.819441             -0.823590   
1          0.907389           -1.380216              0.924293   
2         -0.701950            0.027437             -0.124437   
3         -1.506619            0.254838             -1.023349   
4          1.597106            1.958951              0.374958   

   Weekday Reservations   Revenue  Marketing&Followers  Location_Downtown  \
0             -1.261571 -0.064043            -0.870544              False   
1             -1.161586 -0.620285             0.712762               True   
2             -0.761648 -0.428956            -0.122085              False   
3             -0.611671 -0.940598            -1.591078              False   
4             -0.161740  3.122598             0.314123               True   

   Location_Rural  Location_Suburban  Cuisine_American  Cuisine_French  \
0            True              False             False           False  

Summarizing, you must first scale (or standardize) separately each input variable
of your data, scale your desired output variables, and use these scaled patterns to train
the neural network. Then, when the training is finished, you can make predictions over
new patterns in three steps: scale the pattern, introduce it into the neural network,
and descale the obtained output from the network.

Import out in-house neural network class, at the beginning it will output a lot of internal debugging immediately

In [ ]:
from NeuralNet import NeuralNet

In [ ]:
layers = [4, 9, 5, 1]
nn = NeuralNet(layers=layers)

print("L = ", nn.L, end="\n")
print("n = ", nn.n, end="\n")

print("xi = ", nn.xi, end="\n")
print("xi[0] = ", nn.xi[0], end="\n")
print("xi[1] = ", nn.xi[0], end="\n")

print("wh = ", nn.w, end="\n")
print("wh[1] = ", nn.w[1], end="\n")

L =  4
n =  [4, 9, 5, 1]
xi =  [array([0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]), array([0.])]
xi[0] =  [0. 0. 0. 0.]
xi[1] =  [0. 0. 0. 0.]
wh =  [array([[0.]]), array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0.]])]
wh[1] =  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
    # Optional: Plot the evolution of the training and validation errors
    # Feed−forward all test patterns
    # Descale the predictions of test patterns, and evaluate them
    # test_predictions = self.predict(x)